#Imports

In [72]:
import json
import pydicom
import cv2
import matplotlib.pyplot as plt
import glob
import numpy as np
import os
# json_parse is not an installed module in this environment, and the helper functions
# get_coordinates and get_dicom are already defined in another notebook cell (see CELL INDEX 2).
# Remove the failing import and use those functions directly.




In [ ]:


def get_coordinates(annotation):
    coordinates = []
    x= annotation.get("x")
    y= annotation.get("y")
    coordinates.append((x,y))
    return coordinates


def get_dicom(dicom_id,dicom_view,dicom_laterilty):
    #Change directory to dicom dataset path
    dataset_path = "C:\\Users\\alqud\\Desktop\\2025\\Breast Cancer\\dataset-uta7-dicom\\dataset"
    os.chdir(dataset_path)
    patient_dict= {"6244f5c5-faf5-4e52-83e6-74fee6476dec":"Anonymized1",
                   "829de0c1-5e7a-4b9a-978a-537922a46b35":"Anonymized2",
                   "fb8c2ffc-5f8c-4ac7-a8d9-74b3c56b69a4":"Anonymized3",
                   "b191c30e-db18-4d25-b156-6b6bc2dda995":"Anonymized4"
                   }
    for filename in glob.glob(patient_dict[dicom_id]+"\\*.dcm"): # Loop through all DICOM files in the directory

        print(f"Checking DICOM file: {filename}")
        ds = pydicom.dcmread(filename)
        if ds.get("ViewPosition") == dicom_view and ds.get("ImageLaterality") == dicom_laterilty:
            print(f"Matched DICOM file: {filename}")
            return ds





def create_mask(dicom_image, annotation_data):
    """
    Reads a DICOM and an annotation object, returns the Image and the Binary Mask.
    """
    
    # A. Read the DICOM file
    # If you don't have the real file yet, we can create a dummy image for testing

    ds = dicom_image
    image_data = ds.pixel_array
    height, width = image_data.shape
    # B. Create an empty black mask (same size as image)
    mask = np.zeros((height, width), dtype=np.uint8)

    # C. Extract coordinates
    # We look inside the first item of 'freehand' -> 'handles'
    roi_points = []
    for point in annotation_data:
         print(point[1])
         x = int(point[0]) 
         y = int(point[1])
         roi_points.append([x, y])
    if roi_points:
        # Convert to numpy array of shape (N, 1, 2)
        roi_array = np.array([roi_points], dtype=np.int32)

        # D. Draw the filled polygon onto the mask
        # 255 represents white (the tumor), 0 is black (background)
        cv2.fillPoly(mask, roi_array, 255)

    return image_data, mask

def overlay_mask_on_image(image, mask, alpha=0.4, ax=None, show=True):
            """
            Overlay a binary mask on a grayscale image using a light green color.
            If side_by_side is True, show the original image (left) and the overlay (right).
            Otherwise draw the overlay on a single axis.

            Parameters:
            - image: 2D numpy array (grayscale) or 3D RGB image
            - mask: 2D numpy array with mask pixels >0 indicating the region
            - alpha: float in [0,1], transparency of the green overlay
            - ax: optional Matplotlib Axes (single) to draw on when side_by_side is False
            - show: if True, call plt.show()
            - side_by_side: if True, create two subplots (original, overlay)

            Returns:
            - ax or array of ax: Matplotlib Axes with the overlay plotted
            """
            mask_bool = (mask is not None) and (mask > 0)

        
            fig, axes = plt.subplots(1, 2, figsize=(12, 6))
            ax_orig, ax_overlay = axes
            # Original image
            if image.ndim == 2:
                ax_orig.imshow(image, cmap='gray')
            else:
                ax_orig.imshow(image)
            ax_orig.set_title("Original")
            ax_orig.set_axis_off()

            # Prepare overlay image
            if mask is None:
                # nothing to overlay — just show original again (or blank)
                if image.ndim == 2:
                    ax_overlay.imshow(image, cmap='gray')
                else:
                    ax_overlay.imshow(image)
                ax_overlay.set_title("Overlay (no mask)")
                ax_overlay.set_axis_off()
            else:
                h, w = mask.shape
                overlay = np.zeros((h, w, 4), dtype=float)
                overlay[mask_bool] = [0.0, 1.0, 0.0, alpha]

                if image.ndim == 2:
                    ax_overlay.imshow(image, cmap='gray')
                else:
                    ax_overlay.imshow(image)
                ax_overlay.imshow(overlay)
                ax_overlay.set_title("Overlay (mask)")
                ax_overlay.set_axis_off()

            if show:
                plt.show()
            return axes




    


























# image, mask = create_mask_from_json("path_to_your_image.dcm", json_data)

# # --- VISUALIZATION ---
# fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# # Original Image
# ax[0].imshow(image, cmap='gray')
# ax[0].set_title("Original Image (Simulated)")

# # Generated Mask
# ax[1].imshow(mask, cmap='gray')
# ax[1].set_title("Generated Binary Mask")

# plt.show()

In [85]:



def get_coordinates(annotation):
    coordinates = []
    x= annotation.get("x")
    y= annotation.get("y")
    coordinates.append(x)
    coordinates.append(y)
    return coordinates


def get_dicom(dicom_id,dicom_view,dicom_laterilty):
    #Change directory to dicom dataset path
    dataset_path = "C:\\Users\\alqud\\Desktop\\2025\\Breast Cancer\\dataset-uta7-dicom\\dataset"
    os.chdir(dataset_path)
    patient_dict= {"6244f5c5-faf5-4e52-83e6-74fee6476dec":"Anonymized1",
                   "829de0c1-5e7a-4b9a-978a-537922a46b35":"Anonymized2",
                   "fb8c2ffc-5f8c-4ac7-a8d9-74b3c56b69a4":"Anonymized3",
                   "b191c30e-db18-4d25-b156-6b6bc2dda995":"Anonymized4"
                   }
    for filename in glob.glob(patient_dict[dicom_id]+"\\*.dcm"): # Loop through all DICOM files in the directory

        print(f"Checking DICOM file: {filename}")
        ds = pydicom.dcmread(filename)
        if ds.get("ViewPosition") == dicom_view and ds.get("ImageLaterality") == dicom_laterilty:
            print(f"Matched DICOM file: {filename}")
            return ds



#Change directory to annotation dataset path
dataset_path = "C:\\Users\\alqud\\Desktop\\2025\\Breast Cancer\\dataset-uta7-annotations\\dataset" 
os.chdir(dataset_path)
for json_file in glob.glob("*.json"): # Loop through all JSON files in the directory
    dataset_path = "C:\\Users\\alqud\\Desktop\\2025\\Breast Cancer\\dataset-uta7-annotations\\dataset" 
    os.chdir(dataset_path)
    if json_file not in ["b191c30e-db18-4d25-b156-6b6bc2dda995.json","6244f5c5-faf5-4e52-83e6-74fee6476dec.json","fb8c2ffc-5f8c-4ac7-a8d9-74b3c56b69a4.json","829de0c1-5e7a-4b9a-978a-537922a46b35.json"]:
        continue
    print(f"Loading annotations from: {json_file}")
    with open(json_file, 'r') as f:
        json_data = json.load(f)

        json_stacks= json_data.get("rawData", {}).get("stacks", [])
        for stack in json_stacks:
            if "probes" in stack.keys() or "freehand" in stack.keys():
                annotations = "probes" if "probes" in stack.keys() else "freehand"
                if stack.get("seriesDescription") != "MG":
                    continue
                view_position= stack.get("viewPosition")
                laterality= stack.get("imageSide")
                handles =stack.get(annotations, {})[0].get("handles")
                coordinates= []
                for handle in handles:
                    coordinates.append(get_coordinates(handle))
                    print(get_coordinates(handle))
                dicom_ds= get_dicom(json_data.get("rawData", {}).get("studyId"),view_position,laterality)
                image_data, mask=create_mask(dicom_ds,coordinates)
                overlay_mask_on_image(image_data,mask)
                
                
                #     print(f"Coordinates: {coords}")
            
            
    break


Loading annotations from: 6244f5c5-faf5-4e52-83e6-74fee6476dec.json
[1185.3665679874691, 1695.3158753059897]
[1168.5403175313122, 1682.2287916178675]
[1159.1924006112251, 1661.663374393676]
[1168.5403175313122, 1637.3587904014494]
[1181.6274012194344, 1611.1846230252052]
[1198.4536516755913, 1605.575872873153]
[1213.4103187477308, 1614.92378979324]
[1230.2365692038877, 1628.010873481362]
[1243.3236528920097, 1656.0546242416235]
[1243.3236528920097, 1680.3592082338503]
[1226.4974024358528, 1693.4462919219723]
[1205.931985211661, 1695.3158753059897]
Checking DICOM file: Anonymized1\MG000000.dcm
Checking DICOM file: Anonymized1\MG000001.dcm
Checking DICOM file: Anonymized1\MG000002.dcm
Checking DICOM file: Anonymized1\MG000003.dcm
Matched DICOM file: Anonymized1\MG000003.dcm
1695.3158753059897
1682.2287916178675
1661.663374393676
1637.3587904014494
1611.1846230252052
1605.575872873153
1614.92378979324
1628.010873481362
1656.0546242416235
1680.3592082338503
1693.4462919219723
1695.31587530

NameError: name 'side_by_side' is not defined